# Instalar las dependencias necesarias


In [ ]:
# Instalar las dependencias necesarias
!pip install -U spacy scikit-learn
!pip install ipywidgets
!pip install spacy spacy-transformers
!python -m spacy download es_dep_news_trf
!pip install cupy-cuda12x

# Instalar PyTorch (asegúrate de usar la versión adecuada para tu sistema)
import os
if os.system("nvidia-smi") == 0:  # Detectar si hay GPU disponible
    print("Instalando PyTorch con soporte para GPU...")
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
else:
    print("Instalando PyTorch para CPU...")
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


### 1. Prepara JSON para Spacy
Este script convierte anotaciones de NER (Reconocimiento de Entidades Nombradas) en formato JSON 
a un formato compatible con spaCy para entrenamiento. Divide los datos en conjuntos de entrenamiento 
y validación, y guarda los resultados en archivos binarios.

Cómo usar:
1. Coloca un archivo JSON con las anotaciones en el mismo directorio donde se encuentra este script.
2. Ejecuta el script.
3. Los archivos de salida se guardarán en una carpeta llamada `annotations_dataset`.
4. Si ocurre algún error durante la conversión, se registrará en un archivo de log llamado `train_file.txt`.
""" 

In [ ]:
import spacy
from spacy.tokens import DocBin
import json
from sklearn.model_selection import train_test_split
import os

def get_spacy_doc(file, data, nlp):
    """
    Convierte los datos en formato spaCy.
    :param file: Archivo de log para registrar errores.
    :param data: Datos de anotaciones.
    :param nlp: Modelo de lenguaje spaCy.
    :return: Objeto DocBin con los datos procesados.
    """
    db = DocBin()
    for text_data in data:
        try:
            text = text_data[0]
            doc = nlp(text)
            ents = []
            for start, end, label in text_data[1]['entities']:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    ents.append(span)
            doc.ents = ents
            db.add(doc)
        except Exception as e:
            file.write(f"Error procesando el texto: {text}\nError: {str(e)}\n\n")
    return db

# Cargar el modelo de lenguaje en español
nlp = spacy.blank("es")

# Buscar automáticamente un archivo JSON en el directorio actual
json_files = [f for f in os.listdir(os.getcwd()) if f.endswith(".json")]
if not json_files:
    print("No se encontró ningún archivo JSON en el directorio actual.")
    exit()

# Seleccionar el primer archivo JSON encontrado
input_file = json_files[0]
print(f"Archivo JSON encontrado: {input_file}")

# Cargar las anotaciones desde el archivo JSON
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Obtener la lista de anotaciones
annotations = data['annotations']

# Dividir los datos en conjuntos de entrenamiento y validación
train_data, test_data = train_test_split(annotations, test_size=0.2, random_state=42)

# Crear los directorios de salida si no existen
os.makedirs("annotations_dataset", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Generar nombres relevantes para los archivos
log_file_path = os.path.join("logs", "train_file_log.txt")
train_output_path = os.path.join("annotations_dataset", "train.spacy")
valid_output_path = os.path.join("annotations_dataset", "valid.spacy")

# Abrir el archivo de log para registrar errores
with open(log_file_path, 'w') as error_file:
    # Crear y guardar los datos de entrenamiento
    print(f"Convirtiendo {len(train_data)} ejemplos de entrenamiento...")
    train_db = get_spacy_doc(error_file, train_data, nlp)
    train_db.to_disk(train_output_path)
    
    # Crear y guardar los datos de validación
    print(f"Convirtiendo {len(test_data)} ejemplos de validación...")
    test_db = get_spacy_doc(error_file, test_data, nlp)
    test_db.to_disk(valid_output_path)

print("¡Finalizado! Se crearon:")
print(f"- Datos de entrenamiento: {len(train_data)} ejemplos")
print(f"- Datos de validación: {len(test_data)} ejemplos")
print(f"Archivos guardados en la carpeta 'annotations_dataset/'.")
print(f"Archivo de log guardado en: {log_file_path}")

### Verifica anotaciones antes de entrenar

In [ ]:
!python -m spacy debug data config.cfg --paths.train annotations_dataset/train.spacy --paths.dev annotations_dataset/valid.spacy

### Ejecuta entrenamiento
### Training Parameters (config.cfg)

- `--max-epochs`: Maximum number of training epochs
- `--batch-size`: Batch size for training
- `--eval-frequency`: How often to evaluate on validation set
- `--patience`: Early stopping patience
- `--dropout`: Dropout rate during training

## Output

The training will create:
- `output/model-best`: Best model based on validation scores
- `output/model-last`: Last model state
- Training logs with metrics


In [ ]:
!python -m spacy train config.cfg --output ./output --gpu-id 0 --paths.train annotations_dataset/train.spacy --paths.dev annotations_dataset/valid.spacy